In [ ]:
from yahoo_fin import stock_info
from sklearn.preprocessing import MinMaxScaler
from collections import deque
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
import numpy as np
import time
import datetime
import matplotlib.pyplot as plt

In [ ]:
#
#
#
####  Edit these variables as desired   ####
#
DAYS_PER_SEQUENCE = 10  # use the previous x days as params each training iteration to reduce impact of noise
DAYS_AHEAD_PRED = 1     # 0 = td, 1 = tmrw, ...
STOCK_TICKER = 'META'   # stock ticker (can be anything within yahoo fin's database)
#
#############################################
#
#
#

In [ ]:
def PricePredNN(x_train, y_train):
  model = Sequential()
  model.add(LSTM(64, return_sequences=True, input_shape=(DAYS_PER_SEQUENCE, 1)))    # LSTM layer of 64 nodes: return all
  model.add(Dropout(0.2))   # drop 20% (mitigate overfitting)
  model.add(LSTM(128, return_sequences=False))      # LSTM layer of 128 nodes
  model.add(Dropout(0.2))   # drop 20% (mitigate overfitting)
  model.add(Dense(32))      # fully connected layer of 32 nodes
  model.add(Dense(16))      # fully connected layer of 16 nodes
  model.add(Dense(1))      # fully connected layer of 1 node (output)
  epochs = 100
  batch_size = 8

  model.compile(loss='mean_squared_error', optimizer='adam')
  model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1)
  model.summary()
  return model

In [ ]:
def PrepareData(N):
  data_c = data.copy()
  data_c['future'] = data_c['adjclose'].shift(-N)    # create 'future' col (by shifting up adjclose by 'N' days) ['N' is 1,2, or 3]
  sequ_last = np.array(data_c[['adjclose']].tail(N))   # mini array of price at last 'N' days
  data_c.dropna(inplace=True)   # delete any NANs
  sequ_data = []
  sequ = deque(maxlen=DAYS_PER_SEQUENCE)  # each sequence holds data from prev 5 days

  for entry, target in zip(data_c[['adjclose'] + ['date']].values, data_c['future'].values):
      sequ.append(entry)   # add price/date combo to seq
      if len(sequ) == DAYS_PER_SEQUENCE:              # every 5 iterations...
          sequ_data.append([np.array(sequ), target])   # add tuple of ((date/price of  prev N days), 1 target)

  sequ_last = list([s[:len(['adjclose'])] for s in sequ]) + list(sequ_last)   # gets last sequence for final guess
  sequ_last = np.array(sequ_last).astype(np.float32)    # convert to float

  X, Y = [], []
  for seq, target in sequ_data:
      X.append(seq)     # split prev N days into x
      Y.append(target)  # split targets into y

  X = np.array(X)
  Y = np.array(Y)

  return sequ_last, X, Y

In [ ]:
date_td = time.strftime('%Y-%m-%d') # get current date
date_1ya = (datetime.date.today() - datetime.timedelta(days=365)).strftime('%Y-%m-%d')  # date 1yr ago
date_1ya

In [ ]:
# get data from yahoo finance
data_raw = stock_info.get_data( STOCK_TICKER, start_date=date_1ya, end_date=date_td, interval='1d') # from yahoo fin
data = data_raw.drop(['open', 'high', 'low', 'ticker', 'volume', 'close'], axis=1) # remove extra columns (only focusing on adjusted close: closing price after adjustments for all applicable splits and dividend distributions)
data['date'] = data.index # make col for date (instead of just as index)

data_raw

In [ ]:
data

In [ ]:
# plot adjclose
plt.figure(figsize=(10,6))
plt.plot(data['adjclose'][-365:])   # plot adjusted closing price (last yr)
plt.ylabel("Price (USD)")
plt.xlabel("Date")
plt.legend([f'Adj Closing Price for {STOCK_TICKER}'])
plt.show()

In [ ]:
# plot adjclose
plt.figure(figsize=(10,6))
plt.plot(data['adjclose'][-7:])   # plot adjusted closing price (last 7 days)
plt.ylabel("Price (USD)")
plt.xlabel("Date")
plt.legend([f'Adj Closing Price for {STOCK_TICKER}'])
plt.show()

In [ ]:
# plot close
plt.figure(figsize=(10,6))
plt.plot(data_raw['close'][-365:])   # plot adjusted closing price (last yr)
plt.ylabel("Price (USD)")
plt.xlabel("Date")
plt.legend([f'Closing Price for {STOCK_TICKER}'])
plt.show()

In [ ]:
# plot volume
plt.figure(figsize=(10,6))
plt.plot(data_raw['volume'][-365:])   # plot adjusted closing price (last yr)
plt.ylabel("Volume")
plt.xlabel("Date")
plt.legend([f'Trading volume for {STOCK_TICKER}'])
plt.show()

In [ ]:
# scale data
scaler = MinMaxScaler() # scale data to be between 0 and 1
data['adjclose'] = scaler.fit_transform(np.expand_dims(data['adjclose'].values, axis=1))
data[:]

In [ ]:
# plot scaled adjclose
plt.figure(figsize=(10,6))
plt.plot(data['adjclose'][-365:])   # plot scaled adjusted closing price (last yr)
plt.ylabel("Scaled Price")
plt.xlabel("Date")
plt.legend([f'Adj Closing Price for {STOCK_TICKER}'])
plt.show()

In [ ]:
last_sequence, x_train, y_train = PrepareData(DAYS_AHEAD_PRED+1)  # create parameters for each price (so model can base predction while 'looking back' X days)
x_train = x_train[:, :, :len(['adjclose'])].astype(np.float32)    # omit date, only use price
model = PricePredNN(x_train, y_train)

In [ ]:
# note: retrain model (run the above cell) to predict again
pred_sequence = np.expand_dims(last_sequence[-DAYS_PER_SEQUENCE:], axis=0)  # gets last sequence of data (including price we want to find)
prediction = model.predict(pred_sequence)   # predicts price using last sequence of data as params
predicted_price = round(float(scaler.inverse_transform(prediction)[0][0]), 2)   # unscale value

In [ ]:
# print out result
last_price = data_raw['adjclose'].iloc[-1]
print(f"{STOCK_TICKER} price prediction:")
if DAYS_AHEAD_PRED == 1: print(f"In {DAYS_AHEAD_PRED} day: ${predicted_price}")
else: print(f"In {DAYS_AHEAD_PRED} days: ${predicted_price}")
print(f"Change of {round(((predicted_price-last_price)/last_price)*100,2)}%")